In [1]:
"""
UCF50 Video Classification using Recurrent Neural Networks
Implements multiple architectures: Single Frame, Early Fusion, Late Fusion, CNN+LSTM, ConvLSTM
"""

import os
import cv2
import numpy as np
import random
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# SET RANDOM SEEDS
# ============================================================================
mySeed = 42
np.random.seed(mySeed)
random.seed(mySeed)
torch.manual_seed(mySeed)
torch.cuda.manual_seed(mySeed)
torch.backends.cudnn.deterministic = True

# ============================================================================
# CONFIGURATION
# ============================================================================
class Config:
    # Data parameters
    DATA_PATH = 'UCF50'  # Update this path
    NUM_FRAMES = 20  # Use first 20 frames
    IMG_SIZE = (112, 112)  # Frame size
    
    # Class range - Change this for each of the 5 runs
    # Run 1: (0, 10), Run 2: (10, 20), Run 3: (20, 30), Run 4: (30, 40), Run 5: (40, 50)
    CLASS_START = 0
    CLASS_END = 10
    
    # Training parameters
    BATCH_SIZE = 8
    EPOCHS = 50
    LEARNING_RATE = 0.001
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Model selection: 'single_frame', 'early_fusion', 'late_fusion', 'cnn_lstm', 'conv_lstm'
    MODEL_TYPE = 'cnn_lstm'

config = Config()

# ============================================================================
# DATASET CLASS
# ============================================================================
class UCF50Dataset(Dataset):
    def __init__(self, video_paths, labels, transform=None, num_frames=20):
        self.video_paths = video_paths
        self.labels = labels
        self.transform = transform
        self.num_frames = num_frames
        
    def __len__(self):
        return len(self.video_paths)
    
    def load_video(self, path):
        """Load first num_frames from video"""
        cap = cv2.VideoCapture(path)
        frames = []
        
        for i in range(self.num_frames):
            ret, frame = cap.read()
            if not ret:
                # If video has fewer frames, repeat last frame
                if len(frames) > 0:
                    frames.append(frames[-1].copy())
                else:
                    # Create blank frame
                    frames.append(np.zeros((config.IMG_SIZE[0], config.IMG_SIZE[1], 3), dtype=np.uint8))
            else:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, config.IMG_SIZE)
                frames.append(frame)
        
        cap.release()
        return np.array(frames)
    
    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.labels[idx]
        
        # Load video frames
        frames = self.load_video(video_path)
        
        # Apply transforms to each frame
        if self.transform:
            frames = np.stack([self.transform(frame) for frame in frames])
        else:
            frames = torch.FloatTensor(frames).permute(0, 3, 1, 2) / 255.0
        
        return frames, label

# ============================================================================
# DATA LOADING FUNCTIONS
# ============================================================================
def load_ucf50_data(data_path, class_start=0, class_end=10):
    """Load video paths and labels for specified class range"""
    data_path = Path(data_path)
    video_paths = []
    labels = []
    class_names = sorted([d.name for d in data_path.iterdir() if d.is_dir()])
    
    # Select classes in range
    selected_classes = class_names[class_start:class_end]
    class_to_idx = {cls: idx for idx, cls in enumerate(selected_classes)}
    
    print(f"Loading classes {class_start} to {class_end-1}: {selected_classes}")
    
    for class_name in selected_classes:
        class_path = data_path / class_name
        for video_file in class_path.glob('*.avi'):
            video_paths.append(str(video_file))
            labels.append(class_to_idx[class_name])
    
    return video_paths, labels, len(selected_classes)

# ============================================================================
# MODEL ARCHITECTURES
# ============================================================================

class SingleFrameCNN(nn.Module):
    """Single Frame Classification - Most common frame class wins"""
    def __init__(self, num_classes):
        super().__init__()
        resnet = models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        self.fc = nn.Linear(512, num_classes)
        
    def forward(self, x):
        # x shape: (batch, num_frames, channels, height, width)
        batch_size, num_frames = x.shape[0], x.shape[1]
        
        # Process each frame independently
        x = x.view(batch_size * num_frames, *x.shape[2:])
        features = self.features(x)
        features = features.view(batch_size, num_frames, -1)
        
        # Classify each frame
        logits = self.fc(features)  # (batch, num_frames, num_classes)
        
        # Average predictions across frames
        output = logits.mean(dim=1)
        return output

class EarlyFusionCNN(nn.Module):
    """Early Fusion - Concatenate frames in channel dimension"""
    def __init__(self, num_classes, num_frames=20):
        super().__init__()
        self.num_frames = num_frames
        
        # Modified first conv to accept num_frames * 3 channels
        self.conv1 = nn.Conv2d(num_frames * 3, 64, kernel_size=7, stride=2, padding=3)
        
        # Rest of ResNet
        resnet = models.resnet18(pretrained=True)
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        self.avgpool = resnet.avgpool
        self.fc = nn.Linear(512, num_classes)
        
    def forward(self, x):
        # x shape: (batch, num_frames, channels, height, width)
        batch_size = x.shape[0]
        
        # Concatenate all frames in channel dimension
        x = x.view(batch_size, -1, *x.shape[3:])  # (batch, num_frames*3, H, W)
        
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

class LateFusionCNN(nn.Module):
    """Late Fusion - Concatenate CNN features before final classification"""
    def __init__(self, num_classes):
        super().__init__()
        resnet = models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        self.fc = nn.Linear(512, num_classes)
        
    def forward(self, x):
        # x shape: (batch, num_frames, channels, height, width)
        batch_size, num_frames = x.shape[0], x.shape[1]
        
        # Process each frame independently
        x = x.view(batch_size * num_frames, *x.shape[2:])
        features = self.features(x)
        features = features.view(batch_size, num_frames, -1)
        
        # Average features across frames (late fusion)
        fused_features = features.mean(dim=1)
        output = self.fc(fused_features)
        return output

class CNNLSTM(nn.Module):
    """CNN + LSTM - Extract CNN features then process with LSTM"""
    def __init__(self, num_classes, hidden_size=256, num_layers=2):
        super().__init__()
        # CNN feature extractor
        resnet = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(resnet.children())[:-1])
        
        # LSTM
        self.lstm = nn.LSTM(
            input_size=512,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.5
        )
        
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # x shape: (batch, num_frames, channels, height, width)
        batch_size, num_frames = x.shape[0], x.shape[1]
        
        # Extract CNN features for each frame
        x = x.view(batch_size * num_frames, *x.shape[2:])
        cnn_features = self.cnn(x)
        cnn_features = cnn_features.view(batch_size, num_frames, -1)
        
        # Process sequence with LSTM
        lstm_out, (h_n, c_n) = self.lstm(cnn_features)
        
        # Use last hidden state
        output = self.fc(h_n[-1])
        return output

class ConvLSTMCell(nn.Module):
    """Convolutional LSTM Cell"""
    def __init__(self, input_channels, hidden_channels, kernel_size):
        super().__init__()
        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.kernel_size = kernel_size
        self.padding = kernel_size // 2
        
        self.conv = nn.Conv2d(
            in_channels=input_channels + hidden_channels,
            out_channels=4 * hidden_channels,
            kernel_size=kernel_size,
            padding=self.padding
        )
        
    def forward(self, x, hidden):
        h_prev, c_prev = hidden
        
        combined = torch.cat([x, h_prev], dim=1)
        gates = self.conv(combined)
        
        # Split gates
        i, f, o, g = torch.split(gates, self.hidden_channels, dim=1)
        
        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)
        
        c_cur = f * c_prev + i * g
        h_cur = o * torch.tanh(c_cur)
        
        return h_cur, c_cur

class ConvLSTM(nn.Module):
    """ConvLSTM - LSTM with convolutional operations"""
    def __init__(self, num_classes, hidden_channels=64):
        super().__init__()
        self.hidden_channels = hidden_channels
        
        # Initial conv to reduce dimensions
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=7, stride=2, padding=3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # ConvLSTM layers
        self.convlstm1 = ConvLSTMCell(32, hidden_channels, kernel_size=3)
        self.convlstm2 = ConvLSTMCell(hidden_channels, hidden_channels, kernel_size=3)
        
        # Final classification
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(hidden_channels, num_classes)
        
    def forward(self, x):
        # x shape: (batch, num_frames, channels, height, width)
        batch_size, num_frames = x.shape[0], x.shape[1]
        
        # Process first frame to get spatial dimensions
        x0 = self.conv1(x[:, 0])
        h, w = x0.shape[2], x0.shape[3]
        
        # Initialize hidden states
        h1 = torch.zeros(batch_size, self.hidden_channels, h, w).to(x.device)
        c1 = torch.zeros(batch_size, self.hidden_channels, h, w).to(x.device)
        h2 = torch.zeros(batch_size, self.hidden_channels, h, w).to(x.device)
        c2 = torch.zeros(batch_size, self.hidden_channels, h, w).to(x.device)
        
        # Process each frame through ConvLSTM
        for t in range(num_frames):
            x_t = self.conv1(x[:, t])
            h1, c1 = self.convlstm1(x_t, (h1, c1))
            h2, c2 = self.convlstm2(h1, (h2, c2))
        
        # Use final hidden state
        output = self.avgpool(h2)
        output = output.view(batch_size, -1)
        output = self.fc(output)
        return output

# ============================================================================
# TRAINING FUNCTIONS
# ============================================================================
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc='Training')
    for frames, labels in pbar:
        frames = frames.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(frames)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({'loss': running_loss/total, 'acc': 100.*correct/total})
    
    return running_loss / len(dataloader), 100. * correct / total

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for frames, labels in tqdm(dataloader, desc='Validation'):
            frames = frames.to(device)
            labels = labels.to(device)
            
            outputs = model(frames)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(dataloader), 100. * correct / total

# ============================================================================
# MAIN TRAINING LOOP
# ============================================================================
def main():
    print(f"Using device: {config.DEVICE}")
    print(f"Processing classes {config.CLASS_START} to {config.CLASS_END-1}")
    print(f"Model type: {config.MODEL_TYPE}")
    
    # Load data
    video_paths, labels, num_classes = load_ucf50_data(
        config.DATA_PATH, 
        config.CLASS_START, 
        config.CLASS_END
    )
    
    # Train-test split
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        video_paths, labels, test_size=0.2, random_state=mySeed, stratify=labels
    )
    
    print(f"Training samples: {len(train_paths)}, Validation samples: {len(val_paths)}")
    
    # Data augmentation
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Create datasets
    train_dataset = UCF50Dataset(train_paths, train_labels, train_transform, config.NUM_FRAMES)
    val_dataset = UCF50Dataset(val_paths, val_labels, val_transform, config.NUM_FRAMES)
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2)
    
    # Create model
    if config.MODEL_TYPE == 'single_frame':
        model = SingleFrameCNN(num_classes)
    elif config.MODEL_TYPE == 'early_fusion':
        model = EarlyFusionCNN(num_classes, config.NUM_FRAMES)
    elif config.MODEL_TYPE == 'late_fusion':
        model = LateFusionCNN(num_classes)
    elif config.MODEL_TYPE == 'cnn_lstm':
        model = CNNLSTM(num_classes)
    elif config.MODEL_TYPE == 'conv_lstm':
        model = ConvLSTM(num_classes)
    else:
        raise ValueError(f"Unknown model type: {config.MODEL_TYPE}")
    
    model = model.to(config.DEVICE)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5)
    
    # Training loop
    best_acc = 0.0
    for epoch in range(config.EPOCHS):
        print(f'\nEpoch {epoch+1}/{config.EPOCHS}')
        
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, config.DEVICE)
        val_loss, val_acc = validate(model, val_loader, criterion, config.DEVICE)
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        
        scheduler.step(val_acc)
        
        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
            }, f'best_model_{config.MODEL_TYPE}_classes_{config.CLASS_START}_{config.CLASS_END}.pth')
            print(f'Saved best model with accuracy: {best_acc:.2f}%')
    
    print(f'\nTraining completed! Best validation accuracy: {best_acc:.2f}%')
    return best_acc

if __name__ == '__main__':
    main()

Using device: cuda
Processing classes 0 to 9
Model type: cnn_lstm
Loading classes 0 to 9: ['BaseballPitch', 'Basketball', 'BenchPress', 'Biking', 'Billiards', 'BreastStroke', 'CleanAndJerk', 'Diving', 'Drumming', 'Fencing']
Training samples: 1104, Validation samples: 276


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/alex/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:05<00:00, 8.05MB/s]



Epoch 1/50


Validation: 100%|██████████| 35/35 [00:10<00:00,  3.28it/s]


Train Loss: 1.8049, Train Acc: 37.86%
Val Loss: 2.0188, Val Acc: 32.25%
Saved best model with accuracy: 32.25%

Epoch 2/50


...

Train Loss: 0.0157, Train Acc: 99.64%
Val Loss: 0.2823, Val Acc: 95.29%
Saved best model with accuracy: 95.29%

Epoch 49/50


Validation: 100%|██████████| 35/35 [00:07<00:00,  4.39it/s]


Train Loss: 0.0396, Train Acc: 98.73%
Val Loss: 0.3519, Val Acc: 93.12%

Epoch 50/50


Validation: 100%|██████████| 35/35 [00:09<00:00,  3.84it/s]

Train Loss: 0.0350, Train Acc: 99.00%
Val Loss: 0.3355, Val Acc: 93.12%

Training completed! Best validation accuracy: 95.29%
